# Sudoku Solver
## What is Sudoku?
 A sudoku puzzle consists of a 9x9 grid where each cell contains an integer 1-9.
 The 9x9 grid is divided into 9x9 subgrids
The solution to the puzzle must satisfy the following conditions.\
    **>Each row contains digits 1-9 without repetition**\
    **>Each column contains digits 1-9 without repetition**\
    **>Each subgrid contains digits 1-9 without repetition**\
The puzzle is given with some of the cells empty.
This python project aims to create an AI algorithm to solve sudoku.

## Approaching the Problem
The complexity of a Sudoku puzzle depends with the number ofempty cells in the grid\
If 41 of 81 cells are empty then there are 941 possible combininations that could be the solution.
A bruteforce algorithm will be feasable.


## The Dataset
The Dataset used in this project was downloaded from **Kaggle**\

Kaggle has a made publicly available a dataset of 9 million Sudoku puzzles and their solutions.



### importing the required libs and the dataset

In [12]:
#importing the reqired libraries and The Dataset

import numpy as np
import pandas as pd
import time
import itertools

#using pandas too import data set
sudokuDf = pd.DataFrame(pd.read_csv('sudoku.csv',nrows=10**2))

In [13]:
sudokuDf.sample(5)

,puzzle,solution
70,0728340690450200080000750200140800008670400032...,1728345699451267386389754213147896528675421932...
30,0013000020790000000206709030009673007500010490...,8613594723791248565246789134129673857532816499...
7,8001349020410960800050700100086050004063100090...,8671349522415967833958726149786254314563182791...
37,0800000900905020000030014080070906300000000016...,5864371921945823677239614582471986358396547216...
15,0053461700000000500008000095029307410700000030...,9253461787862913543418752695629387411745629838...


#### Representing the Data in the standard sudoku 9x9 grid

As we have seen from the sample the puzzle and solutions are represented by a string of 81 digits so we have to reshape them into a standard sudoku puzzle

In [14]:
def shape(sudokuDf):
    for n in range(sudokuDf.shape[0]):
        #shaping the puzzle part
        sudokuDf.iloc[n,0] = np.reshape(list(sudokuDf.puzzle.values[n]),(9,9)).astype(int)
        #shaping the solution part
        sudokuDf.iloc[n,1] = np.reshape(list(sudokuDf.solution.values[n]),(9,9)).astype(int)
    return sudokuDf

**How the first Puzzle looks**

In [15]:
sudokuDf = shape(sudokuDf)
sudokuDf.iloc[1,0]

array([[3, 0, 1, 0, 8, 6, 5, 0, 4],
       [0, 4, 6, 5, 2, 1, 0, 7, 0],
       [5, 0, 0, 0, 0, 0, 0, 0, 1],
       [4, 0, 0, 8, 0, 0, 0, 0, 2],
       [0, 8, 0, 3, 4, 7, 9, 0, 0],
       [0, 0, 9, 0, 5, 0, 0, 3, 8],
       [0, 0, 4, 0, 9, 0, 2, 0, 0],
       [0, 0, 8, 7, 3, 4, 0, 9, 0],
       [0, 0, 7, 2, 0, 8, 1, 0, 3]])

**This is its Solution**

In [16]:
sudokuDf.iloc[1,1]

array([[3, 7, 1, 9, 8, 6, 5, 2, 4],
       [8, 4, 6, 5, 2, 1, 3, 7, 9],
       [5, 9, 2, 4, 7, 3, 8, 6, 1],
       [4, 6, 3, 8, 1, 9, 7, 5, 2],
       [2, 8, 5, 3, 4, 7, 9, 1, 6],
       [7, 1, 9, 6, 5, 2, 4, 3, 8],
       [6, 3, 4, 1, 9, 5, 2, 8, 7],
       [1, 2, 8, 7, 3, 4, 6, 9, 5],
       [9, 5, 7, 2, 6, 8, 1, 4, 3]])

## Implementing checker for solution
This will be a function to check if the solution set is in line with the aforementioned conditions for a solution.

In [17]:
def checkPuzzle(sudokuPuzzle):
    checkRow = all([all([x in sudokuPuzzle[nrow,:]for nrow in range(9) for x in range(1,10) ])])
    checkColumn = all([all([x in sudokuPuzzle[ncol,:]for ncol in range(9) for x in range(1,10) ])])
    checkUpperLeft = all([x in sudokuPuzzle[0:3,0:3] for x in range(1,10)])
    checkUpperMid = all([x in sudokuPuzzle[0:3,3:6] for x in range(1,10)])
    checkUpperRight = all([x in sudokuPuzzle[0:3,6:9] for x in range(1,10)])

    checkMidLeft = all([x in sudokuPuzzle[3:6,0:3] for x in range(1,10)])
    checkMidMid = all([x in sudokuPuzzle[3:6,3:6] for x in range(1,10)])
    checkMidRight = all([x in sudokuPuzzle[3:6,6:9] for x in range(1,10)])

    checkLowerLeft = all([x in sudokuPuzzle[6:9,0:3] for x in range(1,10)])
    checkLowerMid = all([x in sudokuPuzzle[6:9,3:6] for x in range(1,10)])
    checkLowerRight = all([x in sudokuPuzzle[6:9,6:9] for x in range(1,10)])

    solved = all([checkRow,checkColumn,checkUpperLeft,checkUpperMid,checkUpperRight,
                  checkMidLeft,checkMidMid,checkMidRight,checkLowerLeft,checkLowerMid,checkLowerRight])
    if solved:
        for line in sudokuPuzzle:
            print(*line)
    return solved

## Using the brute force approach
We can define a function that will scan through each cell and determines which values are already in the cell's row,column and subgrid.\
it then removes known values from a list of values from 1-9 \
if the cell is already filled there can only be a single value the given value

In [18]:
def determineValues(sudokuPuzzle):
    puzzleValue = list()
    for c in range(9):
        for r in range(9):
            if sudokuPuzzle[r,c]==0:
                cellValues = np.array(range(1,10))
                cellValues= np.setdiff1d(cellValues,sudokuPuzzle[r,:]
                                         [np.where(sudokuPuzzle[r,:]!=0)]).tolist()
                cellValues =  np.setdiff1d(cellValues,sudokuPuzzle[:,c]
                                         [np.where(sudokuPuzzle[:,c]!=0)]).tolist()
                puzzleValue.append(cellValues)
            else:
                cellValues= [sudokuPuzzle[r,c]]
                puzzleValue.append(cellValues)
    return puzzleValue       

In [19]:
puzzleValue = determineValues(sudokuDf.iloc[3,0])
puzzleValue

[[2, 5, 6],
 [6, 7],
 [1, 2, 5, 6],
 [3],
 [9],
 [1, 2, 6],
 [2, 5, 7],
 [8],
 [4],
 [5, 6, 9],
 [6, 8],
 [5, 6, 8, 9],
 [2],
 [6, 8],
 [4],
 [3],
 [7],
 [1],
 [8],
 [4],
 [3, 9],
 [7],
 [1],
 [5],
 [9],
 [2],
 [6],
 [3],
 [2],
 [5, 8, 9],
 [1],
 [4],
 [7],
 [5, 8, 9],
 [6],
 [5, 9],
 [1],
 [3, 8],
 [4],
 [6],
 [5],
 [2, 3, 9],
 [2, 8, 9],
 [3, 9],
 [7],
 [7],
 [5],
 [2, 3, 6, 8, 9],
 [8],
 [2, 3, 6, 8],
 [2, 3, 6, 9],
 [1],
 [4],
 [2, 3, 9],
 [2, 4, 5, 6],
 [1],
 [2, 3, 5, 6],
 [9],
 [2, 3, 6, 7],
 [8],
 [2, 4, 5, 7],
 [3, 5],
 [2, 3, 5],
 [2, 4, 5, 9],
 [3],
 [7],
 [5],
 [2, 3],
 [1, 2, 3, 9],
 [6],
 [1, 3, 5, 9],
 [8],
 [2, 5, 6],
 [9],
 [1, 2, 3, 5, 6, 8],
 [4],
 [2, 3, 6, 7, 8],
 [1, 2, 3, 6],
 [2, 5, 7, 8],
 [1, 3, 5],
 [2, 3, 5]]

Since we now know the possible values of each cell this we can then move on to to trying out different combinations <br> till we find the solution set but this computational mammoth of a task and requires bot alot of time and resources a better method will be to use backtracking

### implementing backtracking
The bruteforce approach is is too costly therefore we use backtracking which is a variant of depth first search <br>
In backtracking, only one successor is generated at a time rather
than all successors; each partially expanded node remembers which successor to generate
next.
One cell is expounded and we pick a value from our list check validity conditions if true move to another cell if false pick the next number from the list
this is reapeated until all values have have failed the validity check or all cells are fillled
if a second cell c2 fails all test from the list of values then we move to the first cell c1 and pick a different value but firs wee need a function that efficiently checks the subgrid of the current cell

In [20]:
def gridChecker(r,c,sudokuPuzzle,n):
    if r < 3:
        if c<3:
            subgrid = n in sudokuPuzzle[0:3,0:3]
        elif c < 6:
            subgrid = n in sudokuPuzzle[0:3,3:6]
        else:
            subgrid = n in sudokuPuzzle[0:3,6:9]
    elif r < 6:
        if c<3:
            subgrid = n in sudokuPuzzle[3:6,0:3]
        elif c < 6:
            subgrid = n in sudokuPuzzle[3:6,3:6]
        else:
            subgrid = n in sudokuPuzzle[3:6,6:9]
    else:
        if c<3:
            subgrid = n in sudokuPuzzle[6:9,0:3]
        elif c < 6:
            subgrid = n in sudokuPuzzle[6:9,3:6]
        else:
            subgrid = n in sudokuPuzzle[6:9,6:9]
    return subgrid

Implementing the code

In [21]:
def solve(sudokuPuzzle,puzzleValue):
    c =int(0)
    solution = False
    rows = np.array(np.where(sudokuPuzzle == 0))[0]
    cols = np.array(np.where(sudokuPuzzle == 0))[1]
    dic = dict(zip(list(range(len(rows))), np.zeros(len(rows),dtype = int).tolist()))
    while solution == False:
        if c >= len(rows):
            solution = checkPuzzle(sudokuPuzzle)
            break
        r = rows[c]
        c = cols[c]
        len_num = len(np.array(puzzleValue).reshape(9,9)[r,c])
        num = dic[c]
        while num < len_num:
            cell = np.array(puzzleValue).reshape(9,9)[r,c][num]
            checkRow = cell in sudokuPuzzle[r,:]
            if checkRow:
                num += 1
                continue
            checkCol = cell in sudokuPuzzle[:,c]
            if checkCol:
                num += 1
                continue
            checkGrid = gridChecker(r,c,sudokuPuzzle,cell)
            if checkGrid:
                num += 1
                continue
            dic[c] = num
            sudokuPuzzle[r,c] = cell
            c += 1
            break
        else:
            sudokuPuzzle[r,c] = 0 
            dic[c] = 0
            c -= 1



**The above function:** 

Checks the validity of each row, column, and subgrid:<br>
If it fails, the next value in puzzle_value[r,c] is chosen<br>
If it passes, the value is assigned to the cell and we move on to the next cell (and count increases by 1)<br>
If none of the values for a given cell pass (i.e. else is executed): 
The cell value is reset to 0<br>
The loop through the potential values is reset<br>
We move back to the previous cell (and count decreases by one). <br>
To run the code on the first puzzle, we run:

In [22]:
solve(sudokuDf.iloc[3,0],determineValues(sudokuDf.iloc[3,0]))

C:\Users\Jude Tulel\AppData\Local\Temp\ipykernel_5124\2374377685.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  len_num = len(np.array(puzzleValue).reshape(9,9)[r,c])
C:\Users\Jude Tulel\AppData\Local\Temp\ipykernel_5124\2374377685.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cell = np.array(puzzleValue).reshape(9,9)[r,c][num]
